In [1]:
import pandas as pd
import toml
from pathlib import Path

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.0%}'.format)


In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

In [3]:


# list of equity geographies
equity_geogs = summary_config['hh_equity_geogs']
# not_equity_geogs = ["NOT in " + item for item in equity_geogs]

# Household VMT

In [4]:
# vmt data
df_vmt = pd.read_csv(output_path / 'agg/dash/person_vmt.csv')

# add home RGC
df_vmt['is_rgc'] = 'Not in RGC'
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)].copy()

df_hh = pd.read_csv(output_path / 'agg/dash/hh_geog.csv')
df_hh['is_rgc'] = 'Not in RGC'
df_hh.loc[df_hh['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

def vmt_per_hh(df_vmt, df_hh, geog, map=False):
    _df_vmt = df_vmt.groupby(geog).sum()[['travdist_wt']]
    df_hh = df_hh.groupby(geog).sum()[['hhexpfac']]

    df = _df_vmt.merge(df_hh, left_index=True, right_index=True)

    if map:
        df.index = df.index.astype('int').map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })
    
    
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Household'] = df['travdist_wt']/df['hhexpfac']

    
    return df[['Average Miles per Household']]


In [5]:
pd.set_option('display.float_format', '{:,.1f}'.format)
vmt_per_hh(df_vmt, df_hh, "hh_county")

,Average Miles per Household
hh_county,
King,32.1
Kitsap,34.9
Outside Region,68.7
Pierce,39.9
Snohomish,43.7
Region,36.0


In [6]:
vmt_per_hh(df_vmt, df_hh, "is_rgc")

,Average Miles per Household
is_rgc,
In RGC,10.4
Not in RGC,38.7
Region,36.0


In [7]:
vmt_per_hh(df_vmt, df_hh, "hh_rgc")

,Average Miles per Household
hh_rgc,
Auburn,25.0
Bellevue,11.6
Bothell Canyon Park,35.3
Bremerton,13.7
Burien,25.3
Everett,15.4
Federal Way,28.0
Greater Downtown Kirkland,21.7
Kent,22.2


In [8]:
pd.set_option('display.float_format', '{:,.1f}'.format)
vmt_per_hh(df_vmt, df_hh, "hh_rg_proposed")

,Average Miles per Household
hh_rg_proposed,
Cities and Towns,49.9
Core,36.0
HCT,39.2
Metro,21.5
Rural,60.3
Urban Unincorporated,46.5
Region,36.0


In [9]:
df = pd.DataFrame()
for name, col in {
    "People of Color": "hh_efa_poc",
    "Income": "hh_efa_pov200",
    "LEP": "hh_efa_lep",
    "Disability": "hh_efa_dis",
    "Older Adults": "hh_efa_older",
    "Youth": "hh_efa_youth"
}.items():
    df[name] = vmt_per_hh(df_vmt, df_hh, col, map=True)
df

,People of Color,Income,LEP,Disability,Older Adults,Youth
hh_efa_poc,,,,,,
Below Regional Average,38.6,38.7,36.3,37.2,34.6,29.5
Above Regional Average,33.0,34.0,36.2,36.2,38.0,42.4
Higher Share of Equity Population,32.9,28.6,34.6,31.4,36.6,48.0
Region,36.0,36.0,36.0,36.0,36.0,36.0


# Delay

In [10]:
df = pd.read_csv(output_path / 'agg/dash/trip_time_total.csv')
df = df[(df['mode'].isin(['SOV','HOV2','HOV3+'])&(df['dorp']==1))]

In [11]:
pd.options.display.float_format = '{:0,.1f}'.format
# Hours of delay for households in these locations
# df[['Total Delay Hours']]



def delay_per_hh(geog):

    df_hh = pd.read_csv(output_path / 'agg/dash/hh_geog.csv')
    df_hh['is_rgc'] = 'Not in RGC'
    df_hh.loc[df_hh['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

    df = pd.read_csv(output_path / 'agg/dash/trip_time_total.csv')
    df['is_rgc'] = 'Not in RGC'
    df.loc[df['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'
    df = df[(df['mode'].isin(['SOV','HOV2','HOV3+'])&(df['dorp']==1))]
    df = df.groupby(geog).sum()[['travtime_wt']]

    df2 = pd.read_csv(output_path / 'agg/dash/trip_sov_ff_time.csv')
    df2['is_rgc'] = 'Not in RGC'
    df2.loc[df2['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'
    df2 = df2[(df2['mode'].isin(['SOV','HOV2','HOV3+'])&(df2['dorp']==1))]
    df2 = df2.groupby(geog).sum()[['sov_ff_time_wt']]
    df = df2.merge(df, on=geog)

    # Hours of delay from travel time
    df['Total Delay Hours'] = (df['travtime_wt'] - df['sov_ff_time_wt'])/60
    # Set any negative delay to 0
    df.loc[df['Total Delay Hours'] < 0, 'Total Delay Hours'] = 0

    df_hh = df_hh.groupby(geog).sum()[['hhexpfac']]

    df = df.merge(df_hh, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Minutes of Delay per HH'] = df['Total Delay Hours']/df['hhexpfac']*60

    df['Annual Hours of Delay per HH'] = df['Average Minutes of Delay per HH']*summary_config['weekday_to_annual']/60

    df[['Total Delay Hours',
        'Annual Hours of Delay per HH']] = df[['Total Delay Hours', 'Annual Hours of Delay per HH']].astype(int).map('{:,}'.format)


    return df[['Total Delay Hours','Average Minutes of Delay per HH','Annual Hours of Delay per HH']]

df = delay_per_hh('hh_county')
df

,Total Delay Hours,Average Minutes of Delay per HH,Annual Hours of Delay per HH
hh_county,,,
King,"101,538",6.4,33
Kitsap,"4,985",2.7,14
Outside Region,0,3.3,17
Pierce,"35,124",6.0,32
Snohomish,"45,348",8.5,45
Region,"186,997",6.5,34


In [12]:
delay_per_hh('is_rgc')


,Total Delay Hours,Average Minutes of Delay per HH,Annual Hours of Delay per HH
is_rgc,,,
In RGC,"5,022",1.8,9
Not in RGC,"181,974",7.0,37
Region,"186,997",6.5,34


In [13]:
delay_per_hh('hh_rgc')


,Total Delay Hours,Average Minutes of Delay per HH,Annual Hours of Delay per HH
hh_rgc,,,
Auburn,61,3.7,19
Bellevue,425,2.6,13
Bothell Canyon Park,57,12.0,64
Bremerton,62,2.3,12
Burien,134,4.1,21
Everett,132,2.1,11
Federal Way,10,2.8,14
Greater Downtown Kirkland,440,5.9,31
Kent,68,4.2,22


In [14]:
delay_per_hh('hh_rg_proposed')


,Total Delay Hours,Average Minutes of Delay per HH,Annual Hours of Delay per HH
hh_rg_proposed,,,
Cities and Towns,"18,080",8.0,42
Core,"47,073",7.3,38
HCT,"49,724",8.3,44
Metro,"39,866",4.1,21
Rural,"23,690",7.0,37
Urban Unincorporated,"8,561",8.3,44
Region,"186,997",6.5,34


## Vehicle Ownership by Equity Focus Areas

In [15]:
# trip data
hh = pd.read_csv(output_path / 'agg/dash/auto_ownership_efa.csv')

# TRIPS
df_hh = hh.copy()
# add home RGC
df_hh['is_rgc'] = 'Not in RGC'
df_hh.loc[df_hh['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

equity_geogs = ['hh_efa_dis', 'hh_efa_older', 'hh_efa_lep', 'hh_efa_pov200', 'hh_efa_poc', 'hh_efa_youth']
df_hh[equity_geogs] = df_hh[equity_geogs].apply(lambda x: x.\
        map({0: 'Below Regional Average', 
             1: 'Above Regional Average', 
             2: 'Higher Share of Equity Population'}))


In [16]:
def stat_by_equity_geog(df, geog, group):
    """
    Function to calculate statistics by equity geography and vehicle ownership.
    """
    # Group by equity geography and vehicle ownership
    df_grouped = df.groupby([geog, group], as_index=False)['hhexpfac'].sum()
    
    # Calculate total households in each equity geography
    total_hh = df.groupby([geog], as_index=False)['hhexpfac'].sum().rename(columns={'hhexpfac': 'total_hh'})
    
    # Merge the grouped data with total households
    df_merged = df_grouped.merge(total_hh, on=geog)
    
    # Calculate percentage of households with the specified vehicle ownership
    df_merged['percentage'] = df_merged['hhexpfac'] / df_merged['total_hh']
    
    return df_merged.pivot(index=geog, columns=group, values='percentage')


In [17]:
stat_by_equity_geog(df_hh, 'hh_efa_poc', 'hhvehs')

hhvehs,0,1,2,3,4
hh_efa_poc,,,,,
Above Regional Average,0.1,0.3,0.4,0.1,0.1
Below Regional Average,0.1,0.3,0.4,0.2,0.1
Higher Share of Equity Population,0.1,0.4,0.3,0.1,0.1


In [18]:
stat_by_equity_geog(df_hh, 'hh_efa_pov200', 'hhvehs')

hhvehs,0,1,2,3,4
hh_efa_pov200,,,,,
Above Regional Average,0.1,0.3,0.4,0.1,0.1
Below Regional Average,0.1,0.3,0.4,0.2,0.1
Higher Share of Equity Population,0.1,0.4,0.3,0.1,0.1


In [19]:
stat_by_equity_geog(df_hh, 'hh_efa_lep', 'hhvehs')

hhvehs,0,1,2,3,4
hh_efa_lep,,,,,
Above Regional Average,0.1,0.3,0.4,0.2,0.1
Below Regional Average,0.1,0.3,0.4,0.2,0.1
Higher Share of Equity Population,0.1,0.4,0.4,0.1,0.1


In [20]:
stat_by_equity_geog(df_hh, 'hh_efa_dis', 'hhvehs')

hhvehs,0,1,2,3,4
hh_efa_dis,,,,,
Above Regional Average,0.1,0.3,0.4,0.1,0.1
Below Regional Average,0.1,0.3,0.4,0.2,0.1
Higher Share of Equity Population,0.1,0.4,0.3,0.1,0.1


In [21]:
stat_by_equity_geog(df_hh, 'hh_efa_older', 'hhvehs')

hhvehs,0,1,2,3,4
hh_efa_older,,,,,
Above Regional Average,0.1,0.3,0.4,0.2,0.1
Below Regional Average,0.1,0.3,0.4,0.1,0.1
Higher Share of Equity Population,0.1,0.3,0.4,0.2,0.1


In [22]:
stat_by_equity_geog(df_hh, 'hh_efa_youth', 'hhvehs')

hhvehs,0,1,2,3,4
hh_efa_youth,,,,,
Above Regional Average,0.0,0.3,0.4,0.2,0.1
Below Regional Average,0.1,0.4,0.3,0.1,0.1
Higher Share of Equity Population,0.0,0.2,0.4,0.2,0.1
